image-classification

In [1]:
import os
import cv2
import numpy as np
from sklearn.model_selection import train_test_split

def np_CountUpContinuingOnes(b_arr):
    # indice continuing zeros from left side.
    # ex: [0,1,1,0,1,0,0,1,1,1,0] -> [0,0,0,3,3,5,6,6,6,6,10]
    left = np.arange(len(b_arr))
    left[b_arr > 0] = 0
    left = np.maximum.accumulate(left)

    # from right side.
    # ex: [0,1,1,0,1,0,0,1,1,1,0] -> [0,3,3,3,5,5,6,10,10,10,10]
    rev_arr = b_arr[::-1]
    right = np.arange(len(rev_arr))
    right[rev_arr > 0] = 0
    right = np.maximum.accumulate(right)
    right = len(rev_arr) - 1 - right[::-1]

    return right - left - 1


def ExtractBreast(img):
    img_copy = img.copy()
    img = np.where(img <= 20, 0, img)  # To detect backgrounds easily
    height, _ = img.shape

    # whether each col is non-constant or not
    y_a = height // 2 + int(height * 0.4)
    y_b = height // 2 - int(height * 0.4)
    b_arr = img[y_b:y_a].std(axis=0) != 0
    continuing_ones = np_CountUpContinuingOnes(b_arr)
    # longest should be the breast
    col_ind = np.where(continuing_ones == continuing_ones.max())[0]
    img = img[:, col_ind]

    # whether each row is non-constant or not
    _, width = img.shape
    x_a = width // 2 + int(width * 0.4)
    x_b = width // 2 - int(width * 0.4)
    b_arr = img[:, x_b:x_a].std(axis=1) != 0
    continuing_ones = np_CountUpContinuingOnes(b_arr)
    # longest should be the breast
    row_ind = np.where(continuing_ones == continuing_ones.max())[0]

    return img_copy[row_ind][:, col_ind]

# 定义路径
TXT_PATH = '/Volumes/图图/MIAS/archive/Info.txt'
PGM_PATH = '/Volumes/图图/MIAS/archive/all-mias'
OUTPUT_BASE_PATH = '/Volumes/图图/MIAS/archive/image-classification'

# 读取TXT文件
with open(TXT_PATH, 'r') as file:
    lines = file.readlines()

# 跳过标题行
lines = lines[1:]

refnums = [line.split()[0] for line in lines]
unique_refnums = list(set(refnums))

# 将去重后的 refnum 按8:2分为训练集和测试集
train_refnums, test_refnums = train_test_split(unique_refnums, test_size=0.2, random_state=42)

# 根据去重后的 refnum 划分原始数据集
train_lines = [line for line in lines if line.split()[0] in train_refnums]
test_lines = [line for line in lines if line.split()[0] in test_refnums]

def process_and_save(lines, set_type):
    for line in lines:
        parts = line.split()
        refnum = parts[0]
        bg = str(parts[1]).replace(' ','')
        # cls = parts[2]
        # severity = parts[3] if len(parts) > 3 else 'N'

        # 读取PGM文件
        pgm_file = os.path.join(PGM_PATH, refnum + '.pgm')
        if os.path.exists(pgm_file):
            img = cv2.imread(pgm_file, cv2.IMREAD_GRAYSCALE)
            img = ExtractBreast(img)
            img = cv2.normalize(img, None, 0, 255, cv2.NORM_MINMAX).astype(np.uint8)

            # 创建输出目录
            output_dir = os.path.join(OUTPUT_BASE_PATH, set_type, refnum)
            if not os.path.exists(output_dir):
                os.makedirs(output_dir)

            # 保存预处理后的图像为JPEG格式
            jpg_output_path = os.path.join(output_dir, 'img.jpg')
            cv2.imwrite(jpg_output_path, img)

            # 保存标签为NumPy文件字典
            info = {
                'background': bg
                # 'class': cls,
                # 'severity': severity
            }
            npy_path = os.path.join(output_dir, 'info_dict.npy')
            np.save(npy_path, info)

            print(f"Processed {refnum} for {set_type} set")
        else:
            print(f"PGM file for {refnum} not found.")

# 处理并保存训练集和测试集
process_and_save(train_lines, 'Train')
process_and_save(test_lines, 'Test')


Processed mdb001 for Train set
Processed mdb002 for Train set
Processed mdb003 for Train set
Processed mdb006 for Train set
Processed mdb010 for Train set
Processed mdb011 for Train set
Processed mdb012 for Train set
Processed mdb013 for Train set
Processed mdb014 for Train set
Processed mdb015 for Train set
Processed mdb016 for Train set
Processed mdb017 for Train set
Processed mdb018 for Train set
Processed mdb019 for Train set
Processed mdb020 for Train set
Processed mdb021 for Train set
Processed mdb022 for Train set
Processed mdb023 for Train set
Processed mdb024 for Train set
Processed mdb026 for Train set
Processed mdb027 for Train set
Processed mdb028 for Train set
Processed mdb029 for Train set
Processed mdb030 for Train set
Processed mdb031 for Train set
Processed mdb032 for Train set
Processed mdb034 for Train set
Processed mdb036 for Train set
Processed mdb037 for Train set
Processed mdb038 for Train set
Processed mdb040 for Train set
Processed mdb041 for Train set
Processe

cropped-classification

In [3]:
import os
import cv2
import numpy as np
from sklearn.model_selection import train_test_split

def crop_and_save(img, x, y, radius, output_path):
    h,w=img.shape
    # 确保裁剪区域不超出图像边界
    x = int(x)
    y = w-int(y)
    radius = int(radius)
    x1 = max(x - radius, 0)
    y1 = max(y - radius, 0)
    x2 = min(x + radius, img.shape[1])
    y2 = min(y + radius, img.shape[0])

    # 裁剪图像
    cropped_img = img[y1:y2, x1:x2]
    cropped_img = cv2.normalize(cropped_img, None, 0, 255, cv2.NORM_MINMAX).astype(np.uint8)
    
    # 保存裁剪后的图像
    cv2.imwrite(output_path, cropped_img)


# 定义路径
TXT_PATH = '/Volumes/图图/MIAS/archive/Info.txt'
PGM_PATH = '/Volumes/图图/MIAS/archive/all-mias'
OUTPUT_BASE_PATH = '/Volumes/图图/MIAS/archive/cropped-classification'

# 读取TXT文件
with open(TXT_PATH, 'r') as file:
    lines = file.readlines()

# 跳过标题行
lines = lines[1:]
lines = [line for line in lines if len(line.split()) > 4]

train_lines, test_lines = train_test_split(lines, test_size=0.2, random_state=42)

def process_and_save(lines, set_type):
    refnum_counter = {}
    for line in lines:
        # print(line)
        parts = line.split()
        refnum = parts[0]
        # bg = parts[1]
        cls = str(parts[2]).replace(' ','')
        severity = str(parts[3]).replace(' ','')
        x = parts[4]
        y = parts[5]
        radius = parts[6]
        
        if refnum in refnum_counter:
            refnum_counter[refnum] += 1
        else:
            refnum_counter[refnum] = 1
        refnum_with_suffix = f"{refnum}_{refnum_counter[refnum]}"

        # 读取PGM文件
        pgm_file = os.path.join(PGM_PATH, refnum + '.pgm')
        if os.path.exists(pgm_file):
            img = cv2.imread(pgm_file, cv2.IMREAD_GRAYSCALE)
            # img = preprocess(img)
            # img = cv2.normalize(img, None, 0, 255, cv2.NORM_MINMAX).astype(np.uint8)

            # 创建输出目录
            output_dir = os.path.join(OUTPUT_BASE_PATH, set_type, refnum_with_suffix)
            if not os.path.exists(output_dir):
                os.makedirs(output_dir)

            jpg_output_path = os.path.join(output_dir, 'img.jpg')
            crop_and_save(img, x, y, radius, jpg_output_path)

            # 保存标签为NumPy文件字典
            info = {
                # 'background': bg
                'class': cls,
                'severity': severity
            }
            npy_path = os.path.join(output_dir, 'info_dict.npy')
            np.save(npy_path, info)

            print(f"Processed {refnum} for {set_type} set")
        else:
            print(f"PGM file for {refnum} not found.")

# 处理并保存训练集和测试集
process_and_save(train_lines, 'Train')
process_and_save(test_lines, 'Test')


Processed mdb130 for Train set
Processed mdb025 for Train set
Processed mdb032 for Train set
Processed mdb188 for Train set
Processed mdb256 for Train set
Processed mdb090 for Train set
Processed mdb179 for Train set
Processed mdb081 for Train set
Processed mdb115 for Train set
Processed mdb152 for Train set
Processed mdb209 for Train set
Processed mdb102 for Train set
Processed mdb170 for Train set
Processed mdb191 for Train set
Processed mdb019 for Train set
Processed mdb107 for Train set
Processed mdb274 for Train set
Processed mdb184 for Train set
Processed mdb097 for Train set
Processed mdb134 for Train set
Processed mdb213 for Train set
Processed mdb012 for Train set
Processed mdb202 for Train set
Processed mdb181 for Train set
Processed mdb121 for Train set
Processed mdb142 for Train set
Processed mdb111 for Train set
Processed mdb058 for Train set
Processed mdb178 for Train set
Processed mdb110 for Train set
Processed mdb214 for Train set
Processed mdb015 for Train set
Processe

detection

In [4]:
import os
import numpy as np
import cv2
from tqdm import tqdm
from sklearn.model_selection import train_test_split

def find_cropped_image_position(full_image, cropped_image):
    res = cv2.matchTemplate(full_image, cropped_image, cv2.TM_CCOEFF_NORMED)
    min_val, max_val, min_loc, max_loc = cv2.minMaxLoc(res)
    top_left = max_loc
    bottom_right = (top_left[0] + cropped_image.shape[1], top_left[1] + cropped_image.shape[0])
    return [top_left[0], top_left[1], bottom_right[0], bottom_right[1]]

def preprocess_image(img):
    # 去除全为0的行和列
    rows_to_keep = np.any(img != 0, axis=1)
    cols_to_keep = np.any(img != 0, axis=0)
    img = img[rows_to_keep][:, cols_to_keep]
    return img

# 读取txt文件
txt_path = '/Volumes/图图/MIAS/archive/Info.txt'  # 修改为实际的路径
with open(txt_path, 'r') as file:
    lines = file.readlines()

# 跳过标题行
lines = lines[1:]
lines = [line for line in lines if len(line.split()) > 4]

# 分割训练集和测试集
train_lines, test_lines = train_test_split(lines, test_size=0.2, random_state=42)

def process_and_save(lines, set_type, output_base_path):
    data_path = '/Volumes/图图/MIAS/archive/all-mias'  # 图像文件路径
    for line in tqdm(lines):
        parts = line.split()
        img_id = parts[0].strip()
        cls = str(parts[2]).replace(' ', '')
        severity = str(parts[3]).replace(' ', '')
        x = int(parts[4])
        y = int(parts[5])
        radius = int(parts[6])
        
        # 读取图像
        img_path = os.path.join(data_path, f'{img_id}.pgm')
        img = cv2.imread(img_path, cv2.IMREAD_GRAYSCALE)
        if img is None:
            print(f"Image {img_id} not found, skipping.")
            continue
        
        # Crop the region based on x, y, and radius
        try:
            cropped_image = img[y-radius:y+radius, x-radius:x+radius]
        except:
            print(f"Cropping failed for {img_id}, skipping.")
            continue
        
        # 预处理原图像（去除全为0的行和列）
        preprocessed_img = preprocess_image(img)
        
        # 保存数据
        output_dir = os.path.join(output_base_path, set_type, img_id)
        os.makedirs(output_dir, exist_ok=True)
        
        jpg_path = os.path.join(output_dir, 'img.jpg')
        cv2.imwrite(jpg_path, cv2.normalize(preprocessed_img, None, 0, 255, cv2.NORM_MINMAX).astype(np.uint8))
        
        # 在预处理后的图像上找到cropped_image的位置
        bbox = find_cropped_image_position(preprocessed_img, cropped_image)
        
        # 准备bbox和label
        bbox_list = {
            "boxes": [bbox],
            "labels": [cls]
        }
        
        # 检查bboxes.npy文件是否存在
        bbox_output_path = os.path.join(output_dir, 'bboxes.npy')
        if os.path.exists(bbox_output_path):
            existing_data = np.load(bbox_output_path, allow_pickle=True).item()
            existing_data["boxes"].append(bbox)
            existing_data["labels"].append(cls)
            bbox_list = existing_data
        
        # 保存或更新bboxes.npy文件
        np.save(bbox_output_path, bbox_list)
        
        print(f'Saved {img_id} data to {output_dir}')

# 定义保存路径
target_dir = "/Volumes/图图/MIAS/archive/det"

# 处理并保存训练集
process_and_save(train_lines, "Train", target_dir)

# 处理并保存测试集
process_and_save(test_lines, "Test", target_dir)


  1%|          | 1/95 [00:02<03:53,  2.49s/it]

Saved mdb130 data to /Volumes/图图/MIAS/archive/det/Train/mdb130


  2%|▏         | 2/95 [00:05<04:41,  3.03s/it]

Saved mdb025 data to /Volumes/图图/MIAS/archive/det/Train/mdb025


  3%|▎         | 3/95 [00:06<03:04,  2.00s/it]

...

Saved mdb124 data to /Volumes/图图/MIAS/archive/det/Test/mdb124


 96%|█████████▌| 23/24 [00:25<00:01,  1.16s/it]

Saved mdb231 data to /Volumes/图图/MIAS/archive/det/Test/mdb231


100%|██████████| 24/24 [00:26<00:00,  1.09s/it]

Saved mdb148 data to /Volumes/图图/MIAS/archive/det/Test/mdb148
